# 제 7 장 __비지도학습의 이해와 활용__
___

## __사전설정__
---

(1) 저장소 데이터 가져오기

In [ ]:
!rm -rf /content/BizDataAnalysis/

In [ ]:
!git clone https://github.com/BizStat/BizDataAnalysis.git


(2) matplotlib 환경에서 한글 사용

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -f -v
!rm ~/.cache/matplotlib -rf

런타임 메뉴에서 '세션 다시 시작' 후 다음의 명령문 실행

In [ ]:
from matplotlib import rc
rc('font', family='NanumGothicCoding')
rc('axes', unicode_minus=False)

(3) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

___

## 7.2 __군집분석의 이해와 활용__

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### (1) 붓꽃 데이터

* 데이터 가져오기

In [ ]:
iris = pd.read_table('/content/BizDataAnalysis/DATA/iris.csv',sep=',')

In [ ]:
iris.info()

In [ ]:
iris.head()

In [ ]:
X = iris.iloc[:,0:4]

* 계층적 군집분석

In [ ]:
import scipy.cluster.hierarchy as sch

In [ ]:
sch.dendrogram(sch.linkage(X, method='ward'))
plt.show()

In [ ]:
sch.dendrogram(sch.linkage(X, method='median'))
plt.show()

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
ward = AgglomerativeClustering(n_clusters=3, linkage="average")

In [ ]:
ward.fit(X)

In [ ]:
ward.fit_predict(X)

In [ ]:
ward.labels_

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
standard_scaler = StandardScaler()
X_scaled = standard_scaler.fit_transform(X)

In [ ]:
ward.fit_predict(X_scaled)

In [ ]:
pd.crosstab(iris['species'],ward.fit_predict(X_scaled))

In [ ]:
pd.crosstab(iris['species'],ward.fit_predict(X))

* K-means Clustering

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=1)

In [ ]:
kmeans.fit_predict(X_scaled)

In [ ]:
kmeans.labels_

In [ ]:
kmeans.cluster_centers_

In [ ]:
kmeans.feature_names_in_

In [ ]:
pd.crosstab(iris['species'],kmeans.fit_predict(X))

* 군집분석의 성능 평가

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
silhouette_score(X,kmeans.fit_predict(X))

In [ ]:
silhouette_score(X_scaled,ward.labels_)

In [ ]:
silhouette_score(X,kmeans.labels_)

In [ ]:
from sklearn.metrics import silhouette_samples

In [ ]:
silhouette_samples(X,ward.labels_)

In [ ]:
X['cluster'] = ward.labels_
X['score'] = silhouette_samples(X,ward.labels_)

In [ ]:
X.groupby('cluster')['score'].mean()


In [ ]:
#@title 여러개의 클러스터링 갯수를 List로 입력 받아 각각의 실루엣 계수를 면적으로 시각화한 함수 작성
# (출처:https://romg2.github.io/
def visualize_silhouette(cluster_lists, X_features):

    from sklearn.datasets import make_blobs
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_samples, silhouette_score

    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import math

    # 입력값으로 클러스터링 갯수들을 리스트로 받아서, 각 갯수별로 클러스터링을 적용하고 실루엣 개수를 구함
    n_cols = len(cluster_lists)

    # plt.subplots()으로 리스트에 기재된 클러스터링 수만큼의 sub figures를 가지는 axs 생성
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)

    # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 실루엣 개수 시각화
    for ind, n_cluster in enumerate(cluster_lists):

        # KMeans 클러스터링 수행하고, 실루엣 스코어와 개별 데이터의 실루엣 값 계산.
        clusterer = KMeans(n_clusters = n_cluster, max_iter=500, random_state=0)
        cluster_labels = clusterer.fit_predict(X_features)

        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)

        y_lower = 10
        axs[ind].set_title('Number of Cluster : '+ str(n_cluster)+'\n' \
                          'Silhouette Score :' + str(round(sil_avg,3)) )
        axs[ind].set_xlabel("The silhouette coefficient values")
        axs[ind].set_ylabel("Cluster label")
        axs[ind].set_xlim([-0.1, 1])
        axs[ind].set_ylim([0, len(X_features) + (n_cluster + 1) * 10])
        axs[ind].set_yticks([])  # Clear the yaxis labels / ticks
        axs[ind].set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])

        # 클러스터링 갯수별로 fill_betweenx( )형태의 막대 그래프 표현.
        for i in range(n_cluster):
            ith_cluster_sil_values = sil_values[cluster_labels==i]
            ith_cluster_sil_values.sort()

            size_cluster_i = ith_cluster_sil_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.nipy_spectral(float(i) / n_cluster)
            axs[ind].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                                facecolor=color, edgecolor=color, alpha=0.7)
            axs[ind].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10

        axs[ind].axvline(x=sil_avg, color="red", linestyle="--")

In [ ]:
visualize_silhouette([2,3,4,5],X_scaled)
plt.show()

In [ ]:
#@title 여러개의 클러스터링 갯수를 List로 입력 받아 각각의 클러스터링 결과를 시각화
# (출처:https://romg2.github.io/
def visualize_kmeans_plot_multi(cluster_lists, X_features):

    from sklearn.cluster import KMeans
    from sklearn.decomposition import PCA
    import pandas as pd
    import numpy as np

    # plt.subplots()으로 리스트에 기재된 클러스터링 만큼의 sub figures를 가지는 axs 생성
    n_cols = len(cluster_lists)
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)

    # 입력 데이터의 FEATURE가 여러개일 경우 2차원 데이터 시각화가 어려우므로 PCA 변환하여 2차원 시각화
    pca = PCA(n_components=2)
    pca_transformed = pca.fit_transform(X_features)
    dataframe = pd.DataFrame(pca_transformed, columns=['PCA1','PCA2'])

     # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 KMeans 클러스터링 수행하고 시각화
    for ind, n_cluster in enumerate(cluster_lists):

        # KMeans 클러스터링으로 클러스터링 결과를 dataframe에 저장.
        clusterer = KMeans(n_clusters = n_cluster, max_iter=500, random_state=0)
        cluster_labels = clusterer.fit_predict(pca_transformed)
        dataframe['cluster']=cluster_labels

        unique_labels = np.unique(clusterer.labels_)
        markers=['o', 's', '^', 'x', '*']

        # 클러스터링 결과값 별로 scatter plot 으로 시각화
        for label in unique_labels:
            label_df = dataframe[dataframe['cluster']==label]
            if label == -1:
                cluster_legend = 'Noise'
            else :
                cluster_legend = 'Cluster '+str(label)
            axs[ind].scatter(x=label_df['PCA1'], y=label_df['PCA2'], s=70,\
                        edgecolor='k', marker=markers[label], label=cluster_legend)

        axs[ind].set_title('Number of Cluster : '+ str(n_cluster))
        axs[ind].legend(loc='upper right')

    plt.show()

In [ ]:
visualize_kmeans_plot_multi([2,3,4,5],X_scaled)

### 실습 과제

* 데이터 가져오기

In [ ]:
trans = pd.read_excel("/content/BizDataAnalysis/DATA/Online Retail.xlsx")

In [ ]:
trans.info()

In [ ]:
trans.head()

In [ ]:
trans.describe()

* 데이터 정리
  - CustomerID를 정수로 변환

* 날짜 데이터형 처리

* 결측값 및 이상값 제거
  - 하나라도 변수에 결측값이 있는 경우
  - 이상 거래 데이터 : 제품단가, 주문량 등 음수가 존재하는 경우

In [ ]:
sum(trans['CustomerID'].notnull())

In [ ]:
sum(trans['Quantity'] > 0)

* 고객 수 및 기초 통계 산출

In [ ]:
trans.groupby('CustomerID').count()

* 분석 데이터 생성
  - 고객ID별로 최근구매일, 총구매금액, 총구매횟수 등을 합한 데이터로 변환

* RFM분석을 위한 분석 데이터 살펴보기

* 군집분석을 위한 데이터 표준화

* 적정 군집 수를 결정하기 위한 실루엣 계수 산출